# Demonstration of the Gatheral volatility fit

$$ w(k;\chi_R) = a + b \{\rho (k-m) \sqrt{(k-m)^2 + \sigma^2} \} $$

In [40]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
import json
import datetime
from scipy.optimize import curve_fit 

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


import util

In [55]:
def plotResult(logStrike, impVol, a, b, rho, m, sigma):
    
    plt.plot(logStrike, util.rawImpVar(logStrike, a, b, rho, m, sigma), linewidth=3, color='red')
    plt.scatter(logStrike, impVol)
    
    plt.show()    

## Data for simulation

We have to have some real world options data to test the Gatheral model class. From here we can download data or load the pre-saved from the repo.

### Get data from Yahoo finance
In the following code block there is a function from util module that can get options data from Yahoo finance. The Yahoo finance API is changing rapidly so this may not work long after writing this.

In [ ]:
ticker = 'TSLA'
maturity = '2019-11-29'

# Get data from Yahoo finance and save for later testing
data = util.getData(ticker, maturity)
nowString = datetime.datetime.now().strftime('%Y_%m_%d-%H_%M_%S')
fileName = ticker + '__' + nowString
data.to_csv(fileName)

### Load pre-saved data
Because the Yahoo finance API is changing rapidly and pre-saved options data is already in the repo. It is also helpful if there is no net connection to download data.

In [53]:
#df = pd.read_csv('TSLA__2019_11_19-19_29_07')
df = pd.read_csv('TSLA__2019_11_19-20_10_38')

### Get rid of zero implied volatility points

As an artifact of the Yahoo finance API if there is not much activty on a given strike it zeros down the implied volatility of the model which is obviously wring. We filter out these points before fitting.

In [54]:
trh = 10E-4
df = df.drop(df[df['impliedVolatility'] < trh].index)

### Get log-price and implied volitility into two arrays

In [37]:
logStrike = np.log(df['strike'].to_numpy())
impVol = df['impliedVolatility'].to_numpy()

## Fitting parameters and plot fitted model

### Initial guess trick for the fitting 

The idea here is the following. The model class defined by Gatheral can be quite hard to fit if the parameter optimization does not start from closer initial parameters. This is normal in a five parameter non-linear model. The question is how we can init guess the parameters?

There is two low hanging fruit. By examining the model it is clear that "m" is a shift in the basic variable (log-strike) and "a" is the parameter for the minimum value. We can have a good guess for this from the input array by getting the minimum index and the minimum value and use them to init guess the optimization

In [50]:
idx = np.argmin(impVol)
aInit = impVol[idx]
mInit = logStrike[idx]

initParam = [aInit, 0.0, 0.0, mInit, 0.4]

params, params_covariance = curve_fit(util.rawImpVar, logStrike, impVol, p0=initParam)
print(params)

a = params[0]
b = params[1]
rho = params[2]
m = params[3]
sigma = params[4]

[ 0.14553001  1.87925698 -0.31873805  5.77911418  0.11022199]


### Plotting the option data and the fitted model

In [57]:
interact(plotResult, logStrike=fixed(logStrike), impVol=fixed(impVol), a=a, b=b, rho=rho, m=m, sigma=sigma)

interactive(children=(FloatSlider(value=0.14553001095629303, description='a', max=0.4365900328688791, min=-0.1…

<function __main__.plotResult(logStrike, impVol, a, b, rho, m, sigma)>